In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
from vsm import *
from brm import *

In [3]:
b = BRM()
q = b.preprocess_query("(computer OR systems) AND dat*")
q

wildcards: ['dat*']
computer OR systems AND (database|data)


OR(Symbol('computer'), AND(Symbol('systems'), OR(Symbol('database'), Symbol('data'))))

In [4]:
# b.query("crypto* OR security")

In [6]:
v = VSM()
q = "computers graphical"
# v.d_w.loc[:, v.preprocess_query(q)].fillna(0)
# v.query(q)

In [44]:
# bigrams = make_bigrams(v.data.cleaned.tolist())
# bigrams[:3]
p_wi = sum(v.term_dict['language'].values()) #/ sum(map(lambda x: sum(x.values()), v.term_dict.values()))
# p_wi

dict_values([1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [41]:
" ".join(v.data.cleaned).count("language")

22

In [7]:
vocab = v.build_vocab()
list(vocab.items())[0]

('language', ['la', 'an', 'ng', 'gu', 'ua', 'ag', 'ge'])

In [23]:
from spelling import char_ngram
from phrase_indexing import make_bigrams

mis = "langauge"
char_ngram(mis)

['la', 'an', 'ng', 'ga', 'au', 'ug', 'ge']

$$P(la|w_i=\textrm{language}) = \frac{P(la)\cap P(w_i=\textrm{language})}{P(w_i=\textrm{language})}$$

$$P(la|w_i=\textrm{language}) = \frac{P(w_i=\textrm{language}| la) \cdot P(la)}{P(w_i=\textrm{language})}$$

In [ ]:
p_la = []

In [84]:
from phrase_indexing import flatten
from string import ascii_lowercase

letter2index = dict(map(reversed, enumerate(ascii_lowercase)))
print(letter2index)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [91]:
bigrams = flatten(v.build_vocab().values())
bigrams[:3]

['la', 'an', 'ng']

In [90]:
indices = [tuple(map(letter2index.get, x)) for x in bigrams]
indices[:3]

[(11, 0), (0, 13), (13, 6)]

In [149]:
def one_hot(x, dim=26):
    v = np.zeros(dim)
    v[x] = 1
    return v

def log(x):
    return 1 / ( 1 + np.exp(-1 *x))

def d_log(x):
    return log(x) * (1 - log(x))

print(one_hot(6))

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [315]:
Xys = [tuple(map(one_hot, x)) for x in indices]
print(*Xys[0], sep="\n")

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [321]:
X, y = list(zip(*Xys))
X = np.array(X)
y = np.array(y)
print(X[0], y[0], sep="\n")

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [322]:
# np.array(y).shape
# indices[:5]#, "".join(map(index2letter.get, indices[:5]))
# list(map(np.argmax,[s1, s2, s3]))

In [350]:
X.shape

(6777, 26)

In [360]:
wx = np.random.randn()
wrec = np.random.randn()
lr_wx = 0.001
lr_wrec = 0.001

N = X.shape[0]

X, y = list(zip(*Xys))
X = np.array(X); y = np.array(y)

y = np.array(list(map(np.argmax, y)))

In [364]:
state = np.zeros((X.shape[0], X.shape[1] + 1))
state_in = np.zeros((N, N))
grad_over_time = np.zeros((X.shape))

number_of_epoch = 1500
for iter in range(number_of_epoch):
    for i in range(N):
        state_i_in = state[:, i]*wrec + X[:, i]*wx
        state_in[:, i] = state_i_in        
        
        state_i_out = log(state_i_in) if i == (N-1) else state_i_in
        state[:, i+1] = state_i_out

    # cost = np.square(state[:,N] - np.squeeze(y)).sum() / len(X)
    cost = -np.mean(y*state[:, N] + (1-y) * np.log(1-state[:, N]))

    if iter % 1000 == 0: print(f"Current iter: {iter} \t\t Current cost: {cost}")
        
    grad_over_time[:,N-1] = (state[:,N] - np.squeeze(y)) * (2/len(X))
    
    for i in reversed(range(N-1)):
        grad_over_time[:, i] = grad_over_time[:, i+1] * wrec * d_log(state_in[i+1])
    
    grad_wx = np.sum(np.sum([grad_over_time[:, i] * X[:, i] for i in reversed(range(N))]))
    grad_wrec = np.sum(np.sum([grad_over_time[:, i] * state[:, i] for i in reversed(range(N))]))
    
    wx -= lr_wx * grad_wx
    wrec -= lr_wrec * grad_wrec   

IndexError: index 26 is out of bounds for axis 1 with size 26

![img](https://cdn-images-1.medium.com/max/1600/1*s__UUL-tKMDWgg_vkaQvSQ.png)

In [338]:
# np.squeeze(y)
-np.mean(y*state[:, N] + (1-y) * np.log(1-state[:, N]))

-11.408998602326953

In [343]:
y, state[:, N]

(array([ 0, 13,  6, 20]),
 array([0.5       , 0.49999626, 0.5       , 0.5       ]))

In [301]:
for i in range(N):
    state_i_in = state[:, i]*wrec + X[:, i]*wx
    state_in[:, i] = state_i_in        

    state_i_out = log(state_i_in) if i != (N-1) else state_i_in
    state[:, i+1] = state_i_out

In [302]:
print("-----------\n")    
print("Final output Raw: ",state_4_in)
print("Final output: ",np.argmax(state_4_in))
print("Ground Truth : ",np.argmax(y[3]))
print("-----------\n")

-----------

Final output Raw:  [-0.27269297 -0.2730743  -0.27269297 -0.27269297]
Final output:  0
Ground Truth :  20
-----------



In [292]:
y.shape

(4, 26)

In [120]:
alpha, hidden_dim = 0.5, 50
synapse_0 = 2*np.random.random((26,hidden_dim)) - 1
synapse_1 = 2*np.random.random((hidden_dim,26)) - 1

In [135]:
for j in range(1000):
    layer_1 = Stats.sigmoid(np.dot(X,synapse_0))
    layer_2 = Stats.sigmoid(np.dot(layer_1,synapse_1))
    
    layer_2_delta = (layer_2 - y)*(layer_2*(1-layer_2))
    layer_1_delta = layer_2_delta.dot(synapse_1.T) * (layer_1 * (1-layer_1))
    
    synapse_1 -= (alpha * layer_1.T.dot(layer_2_delta))
    synapse_0 -= (alpha * X.T.dot(layer_1_delta))

In [137]:
ht0 = Stats.sigmoid(np.dot(X[0],synapse_0))
ht1 = Stats.sigmoid(np.dot(ht0,synapse_1))

In [142]:
index2letter = dict(map(reversed, letter2index.items()))
print(index2letter)

{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}


In [143]:
list(map(index2letter.get, [np.argmax(X[0]), np.argmax(ht1)]))

['l', 'w']

In [ ]:
import numpy as np

alphas = [0.001,0.01,0.1,1,10,100,1000]

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)
    
X = np.array([[0,0,1],
            [0,1,1],
            [1,0,1],
            [1,1,1]])
                
y = np.array([[0],[1],[1],[0]])

for alpha in alphas:
    print "\nTraining With Alpha:" + str(alpha)
    np.random.seed(1)

    # randomly initialize our weights with mean 0
    synapse_0 = 2*np.random.random((3,4)) - 1
    synapse_1 = 2*np.random.random((4,1)) - 1

    for j in xrange(60000):

        # Feed forward through layers 0, 1, and 2
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0,synapse_0))
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))

        # how much did we miss the target value?
        layer_2_error = layer_2 - y

        if (j% 10000) == 0:
            print "Error after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error)))

        # in what direction is the target value?
        # were we really sure? if so, don't change too much.
        layer_2_delta = layer_2_error*sigmoid_output_to_derivative(layer_2)

        # how much did each l1 value contribute to the l2 error (according to the weights)?
        layer_1_error = layer_2_delta.dot(synapse_1.T)

        # in what direction is the target l1?
        # were we really sure? if so, don't change too much.
        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)

        synapse_1 -= alpha * (layer_1.T.dot(layer_2_delta))
        synapse_0 -= alpha * (layer_0.T.dot(layer_1_delta))

